In [ ]:
import pandas as pd
import pyodbc
import os
import datetime as dt
import shutil

In [ ]:
# retrieve shortage qty from SQL, get unique list of FV/ODM in the past 6 months
# compare with the FV/ODM list on the very day, get the difference list
# amend qty 0 for the different list

In [ ]:
home = os.path.expanduser('~')
today = dt.datetime.today().date()
# retrieve date of 180 days ago
target_date = today - dt.timedelta(days = 180)

In [ ]:
# # connect SQL server
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()
# query statement
# select shortage data for the past 6 months with shortage qty not equal to zero
Query = 'SELECT [ODM],[Item],[Commodity],[FV],[Platform],[HP_PN],[BuyerName] FROM OPS.GPS_tbl_ops_shortage_ext WHERE ReportDate >= ? AND [Total Shortage Qty] != 0'
params = (target_date)
cursor.execute(Query, params)
shortage = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
# shortage.columns = ['ODM','Item','Commodity','FV','Platform','P1','Net P2','Net P3','Total Shortage Qty','BT shortage','Working on upside','ReportDate','last FD date','HP_PN','BuyerName']
cursor.close()
conn.close()

In [ ]:
print(len(shortage))

In [ ]:
shortage['len'] = shortage['HP_PN'].apply(lambda x: len(x))
shortage.sort_values(by = 'len', inplace = True, ascending=True)
shortage.drop(columns = 'len', inplace = True)
shortage = shortage.drop_duplicates(keep = 'last')
print(len(shortage))

In [ ]:
shortage = shortage.drop_duplicates(subset = ['ODM','Commodity','FV'],keep = 'last')
print(len(shortage))
shortage_tmp = shortage[['ODM','Commodity','FV']]

In [ ]:
today_shortage = pd.read_excel(os.path.join(home, 'Desktop', 'shortage_all.xlsx'), sheet_name='Sheet1' )
today_shortage = today_shortage[['ODM','Commodity','FV']]

In [ ]:
merge = shortage_tmp.merge(today_shortage, on = ['ODM','Commodity','FV'], how = 'outer', indicator= True)

In [ ]:
print(len(merge[merge['_merge'] == 'left_only']))
print(len(merge[merge['_merge'] == 'right_only']))
print(len(merge[merge['_merge'] == 'both']))

In [ ]:
amend_data = merge[merge['_merge'] == 'left_only'].drop(columns = '_merge').reset_index(drop = True)
amend_data = amend_data.merge(shortage, on = ['ODM','Commodity','FV'], how = 'left')
# amend_data['Total Shortage Qty'] = 0
amend_data['ReportDate'] = today
amend_data['last FD date'] = today
amend_list = ['P1','Net P2','Net P3','Total Shortage Qty','BT shortage','Working on upside']
for _ in amend_list:
    amend_data[_] = 0


In [ ]:
amend_data.head()

In [ ]:
amend_data.to_excel(os.path.join(home, 'amend_stge_zero.xlsx'))

In [ ]:
# upload amend data to SQL
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()

for index, row in amend_data.iterrows():
    s_ODM = row['ODM']
    s_Item = row['Item']
    s_Commodity = row['Commodity']
    s_FV = row['FV']
    s_Platform = row['Platform']
    s_P1 = row['P1']
    s_P2 = row['Net P2']
    s_P3 = row['Net P3']
    s_Total = row['Total Shortage Qty']
    s_BT = row['BT shortage']
    s_working = row['Working on upside']
    s_ReportDate = pd.to_datetime(row['ReportDate'])
    s_lastFDdate = pd.to_datetime(row['last FD date'])
    s_BuyerName = row['BuyerName']
    s_PN = row['HP_PN']

    cursor.execute(f"INSERT INTO CSI.OPS.GPS_tbl_ops_shortage_ext ( ODM,Item,Commodity,FV,Platform,P1,[Net P2],[Net P3],[Total Shortage Qty],[BT shortage],[Working on upside],ReportDate,[last FD date],HP_PN,BuyerName )\
                    VALUES('{s_ODM}','{s_Item}','{s_Commodity}','{s_FV}','{s_Platform}','{s_P1}','{s_P2}','{s_P3}','{s_Total}','{s_BT}','{s_working}','{s_ReportDate}','{s_lastFDdate}','{s_PN}','{s_BuyerName}')".replace("'NaT'", "NULL"))

conn.commit()
conn.close()

In [ ]:
# 超過180以前等於0的要刪掉，不然會無限長大
# delte shortage = 0 where report date is before 180 days, otherwise, the database will grow up without limit

In [ ]:
# # # connect SQL server
# conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
# cursor = conn.cursor()
# # query statement
# Query = 'DELETE FROM OPS.GPS_tbl_ops_shortage_ext WHERE ReportDate <= ? AND [Total Shortage Qty] = 0'
# params = (target_date)
# cursor.execute(Query, params)
# shortage = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
# # shortage.columns = ['ODM','Item','Commodity','FV','Platform','P1','Net P2','Net P3','Total Shortage Qty','BT shortage','Working on upside','ReportDate','last FD date','HP_PN','BuyerName']
# cursor.close()
# conn.close()